In [2]:
%load_ext autoreload
%autoreload 2

## Calculate zonal means of potential prognostic predictability

In [31]:
import xarray as xr
import numpy as np
import gfdl_utils as gu
import glob
from dask.diagnostics import ProgressBar
from information import *
from processing import *
from variance import *
import os
import re
from xhistogram.xarray import histogram

In [32]:
ppname = 'ocean_cobalt_omip_2d'
frequency = gu.core.get_timefrequency(ppcontrol,ppname)
startmonth = 1
filename = '.'.join([ppname,str(startmonth).zfill(2),'zarr'])
ppp = xr.open_zarr(ppeDict['datasavedir']+'/processed/ppp/'+filename)

In [33]:
grid = xr.open_dataset(ppeDict['griddirtmp']+ppeDict['gridfile'])

In [34]:
masks = generate_masks(grid)

In [35]:
def calc_zonalmean(da,mask,weights,geolat,lat_bins):
    hs_daw = histogram(geolat.where(mask,drop=True),
                             bins=[lat_bins],
                             weights=(da*weights).fillna(0).where(mask,drop=True),
                             dim=['xh','yh'])
    hs_w = histogram(geolat.where(mask,drop=True),
                     bins=[lat_bins],
                     weights=weights.fillna(0).where(mask,drop=True),
                     dim=['xh','yh'])
    return hs_daw/hs_w

In [36]:
weights = grid['areacello']
geolat = grid['geolat']

firstdy = (grid['yq'].values[0]-grid['yh'].values[0])
lat_bins = np.append(grid['yh'].values[0]-firstdy,grid['yq'].values)

In [37]:
savedir = ppeDict['datasavedir']+'/processed/ppp_zonalmeans/'
masknames_global = ['global','atlantic_global','pacific_global','indian_global']
for variable,da in ppp.items():
    for maskname in masknames_global:
        mask = masks[maskname]
        tmp = calc_zonalmean(da,mask,weights,geolat,lat_bins)
        savefilename = '.'.join([variable,frequency,str(startmonth).zfill(2),maskname,'nc'])
        if os.path.isfile(savedir+savefilename):
            print(savefilename+' already exists.')
            continue
        else:
            print('Saving '+savefilename)
            tmp.name=variable
            with ProgressBar():
                tmp.to_netcdf(savedir+savefilename)

Saving intpp.monthly.01.global.nc
[########################################] | 100% Completed | 2.17 sms


/nbhome/gam/miniconda3/envs/predict/lib/python3.11/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


Saving intpp.monthly.01.atlantic_global.nc
[############################            ] | 71% Completed | 1.63 s ms

/nbhome/gam/miniconda3/envs/predict/lib/python3.11/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed | 1.73 s
Saving intpp.monthly.01.pacific_global.nc
[############################            ] | 71% Completed | 1.66 s ms

/nbhome/gam/miniconda3/envs/predict/lib/python3.11/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed | 1.78 s
Saving intpp.monthly.01.indian_global.nc
[########################################] | 100% Completed | 918.16 ms


/nbhome/gam/miniconda3/envs/predict/lib/python3.11/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
